In [47]:
#Develop ticker coverage table
import pandas as pd

TIC = pd.read_csv(r'/Users/JZ/Downloads/TICKER_ALL.csv')

TIC_U = TIC.drop_duplicates(subset=['colummn']).rename(columns={'colummn': 'TickName'})[['TickName']]


In [49]:

INF = pd.read_excel (r'/Users/JZ/Downloads/ALL_INFO.xlsx') 
INF_U = INF.drop_duplicates(subset=['symbol']).rename(columns={'symbol': 'TickName'})[['TickName']]
INF_U['IN_INFO']="In Info Extract"

PRICE = pd.read_csv (r'/Users/JZ/Downloads/ALL_Prices.csv', sep='|')
PRICE_U = PRICE.drop_duplicates(subset=['TickName'])[['TickName']]
PRICE_U['IN_PRICE']="In Price Extract"

FIN = pd.read_csv (r'/Users/JZ/Downloads/ALL_FIN.csv', sep='|')
FIN_U = FIN.drop_duplicates(subset=['TickName'])[['TickName']]
FIN_U['IN_FIN']="In Financial Extract"


In [90]:
# Join it back together
TIC_COVERAGE = TIC_U.merge(INF_U,how='left',on='TickName').merge(PRICE_U,how='left',on='TickName').merge(FIN_U,how='left',on='TickName').fillna('Missing')

TIC_COVERAGE.groupby(['IN_INFO','IN_PRICE','IN_FIN']).count().reset_index().sort_values(by=['TickName'],ascending=False)

,IN_INFO,IN_PRICE,IN_FIN,TickName
0,In Info Extract,In Price Extract,In Financial Extract,3806
1,In Info Extract,In Price Extract,Missing,1742
5,Missing,In Price Extract,Missing,860
7,Missing,Missing,Missing,266
4,Missing,In Price Extract,In Financial Extract,25
3,In Info Extract,Missing,Missing,22
2,In Info Extract,Missing,In Financial Extract,3
6,Missing,Missing,In Financial Extract,1


TickName
IN_INFO         IN_PRICE         IN_FIN                        
In Info Extract In Price Extract In Financial Extract      3806
                                 Missing                   1742
                Missing          In Financial Extract         3
                                 Missing                     22
Missing         In Price Extract In Financial Extract        25
                                 Missing                    860
                Missing          In Financial Extract         1
                                 Missing                    266

In [72]:
TIC_COVERAGE.head(20)

,TickName,IN_INFO,IN_PRICE,IN_FIN
0,MOQ.AX,In Info Extract,In Price Extract,In Financial Extract
1,ONT.AX,In Info Extract,In Price Extract,In Financial Extract
2,14D.AX,In Info Extract,In Price Extract,In Financial Extract
3,1ST.AX,In Info Extract,In Price Extract,In Financial Extract
4,T3D.AX,In Info Extract,In Price Extract,In Financial Extract
5,TDI.AX,In Info Extract,In Price Extract,NaN
6,TGP.AX,In Info Extract,In Price Extract,In Financial Extract
7,TOT.AX,In Info Extract,In Price Extract,In Financial Extract
8,TDO.AX,In Info Extract,In Price Extract,In Financial Extract
9,DDD.AX,In Info Extract,In Price Extract,In Financial Extract
